In [ ]:
import py_aws_vault_auth
import os
import sys
import awswrangler as wr

environ_auth = py_aws_vault_auth.authenticate("sso-sandbox", prompt="python", return_as="environ")
os.environ.update(environ_auth)

### Note: The output of the below queries contains your aws credentials and will be automatically saved in Jupyter history and committed to GitHub, this is not a great idea.
To prevent outputs being commited to GitHub you would need to remember to manually clear all outputs before a commit or install `nbstripout` from pip which will clean out the cells automatically before each commit. 


In [ ]:
!printenv | grep AWS

In [ ]:
!cat ~/.aws/config

In [ ]:
# Retrieve the list of existing buckets
import boto3
s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict({'a':[1], 'b': ['apple']})

In [ ]:
import awswrangler as wr

partition = "2023-05-08" ## "2022-09-22"
athena_merge_partition = f"""
    MERGE INTO sporting_event t
    USING (SELECT op,
    cdc_timestamp,
    id,
    sport_type_name,
    home_team_id,
    away_team_id,
    location_id,
    start_date_time,
    start_date,
    sold_out
    FROM wto_raw_demo.sporting_event_cdc
    WHERE partition_date ='{partition}') s
    ON t.id = s.id
    WHEN MATCHED AND s.op = 'D' THEN DELETE
    WHEN MATCHED THEN
    UPDATE SET
    sport_type_name = s.sport_type_name,
    home_team_id = s.home_team_id,
    location_id = s.location_id,
    start_date_time = s.start_date_time,
    start_date = s.start_date,
    sold_out = s.sold_out
    WHEN NOT MATCHED THEN
    INSERT (id,
    sport_type_name,
    home_team_id,
    away_team_id,
    location_id,
    start_date_time,
    start_date)
    VALUES
    (s.id,
    s.sport_type_name,
    s.home_team_id,
    s.away_team_id,
    s.location_id,
    s.start_date_time,
    s.start_date)
"""
oasys_df = wr.athena.read_sql_query(athena_merge_partition, "wto_curated_demo", workgroup="Athena3", ctas_approach=False)